# Notebook Core Params

In [ ]:
num_split=1 #Which notebook core is this?
num_splits=3 #How many notebooks are running? 
device_num=2 #Which GPU

# Dataset

In [ ]:
import rp
from icecream import ic
from easydict import EasyDict
import copy

In [ ]:
class TestSample:
    def __init__(self, image, prompt:str, path:str, name:str):
        self.path=rp.get_absolute_path(path)
        self.image=image
        self.prompt=prompt
        self.name=name
    
    def copy(self):
        return copy.copy(self)
    
    def display(self):
        print(self.prompt)
        print(self.path)
        rp.display_image(self.image)

In [ ]:
class CroppedRefVOCDataset:
    
    DEFAULT_PROMPT_REPLACEMENTS={
        'aeroplane'    : 'aeroplane',
        'bicycle'      : 'bicycle',
        'bird'         : 'bird',
        'boat'         : 'boat',
        'bottle'       : 'bottle',
        'bus'          : 'bus',
        'car'          : 'car',
        'cat'          : 'cat',
        'chair'        : 'chair',
        'cow'          : 'cow',
        'dog'          : 'dog',
        'horse'        : 'horse',
        'motorbike'    : 'motorcycle',
        'person'       : 'person',
        'potted plant' : 'potted plant',
        'sheep'        : 'sheep',
        'sofa'         : 'sofa',
        'train'        : 'train',
        'tv/monitor'   : 'tv/monitor',
    }
    
    def __init__(self, samples=None, prompt_replacements:dict={}):
        
        #Xiang made this dataset
        dataset_dir='/mnt/md0/nfs/datasets/RefVOC'
        self.dataset_dir = dataset_dir
        
        if samples is None:
            with rp.SetCurrentDirectoryTemporarily(dataset_dir):
                image_names=rp.text_file_to_string('refvoc_files.txt').strip().splitlines()
                #It has a bunch of lines that look like '2008_003885\n2008_004212\n2008_004612\n2008_004621' etc
                image_paths=['cropped-'+x+'.jpg' for x in image_names]
                #These files should exist in the current directory
                images=rp.load_images(image_paths, use_cache=True, show_progress=True)

                assert all(rp.is_image_file(x) for x in image_paths)

                #Get the prompts for every image
                prompts=rp.text_file_to_string('cropped.txt').strip().splitlines()
                #It has a bunch of lines that look like '2010_000241 bird\n2010_000342 bicycle\n2010_000628 car' etc
                prompts=[x.split(maxsplit=1) for x in prompts]
                prompts={image_name:prompt for image_name,prompt in prompts}
                prompts=[prompts[image_name] for image_name in image_names]

                samples=[TestSample(image,prompt,path,name) for image,prompt,path,name in zip(images,prompts,image_paths,image_names)]

        #Replace the given prompts
        prompt_replacements = {**type(self).DEFAULT_PROMPT_REPLACEMENTS, **prompt_replacements}
        prompt_replacements = {x:y for x,y in prompt_replacements.items() if x!=y} #Remove redundancy
                
        for sample in samples:
            if sample.prompt in prompt_replacements:
                sample.prompt = prompt_replacements[sample.prompt]
        self.prompt_replacements=prompt_replacements
        
        self.samples = samples
        
    @property
    def images(self): return [s.image for s in self.samples]
        
    @property
    def prompts(self): return [s.prompt for s in self.samples]

    @property
    def names(self): return [s.name for s in self.samples]

    @property
    def image_paths(self): return [s.image_path for s in self.samples]

    def __getitem__(self, index):
        output = self.samples[index]
        if isinstance(index, int):
            return output
        elif isinstance(index, slice):
            output = CroppedRefVOCDataset(output)
        return output
            
    def split(self, num_divisions, division_index):
        """
        Split the dataset into num_divisions parts and return the division_index part.
        
        num_divisions: An integer representing the number of equal parts to divide the dataset into.
        division_index: An integer representing the 0-indexed part to return.
        """
        division_size = len(self) // num_divisions
        start_index = division_size * division_index
        end_index = start_index + division_size
        if division_index == num_divisions - 1:
            end_index = len(self)
        return self[start_index:end_index]
    
    def __len__(self):
        return len(self.samples)

    def __iter__(self):
        return iter(self.samples)
    
    def __repr__(self):
        return f'CroppedRefVOCDataset(len={len(self)}, prompt_replacements={self.prompt_replacements})'

In [ ]:
cropped_ref_voc_dataset=CroppedRefVOCDataset()
data=cropped_ref_voc_dataset.split(num_splits,num_split)

In [ ]:
data[3:4]

In [ ]:
rp.random_element(data).display()

In [ ]:
rp.random_element(data)

In [ ]:
rp.local_copy(data)

In [ ]:
data.__iter__

# Peekaboo

In [ ]:
import source.stable_diffusion as sd
if 's' not in dir():
    s=sd.StableDiffusion(device=device_num) #Initialize the singleton

In [ ]:
import rp
import source.peekaboo as peekaboo
from source.peekaboo import run_peekaboo
from source.clip import get_clip_logits
from source.stable_diffusion_labels import get_mean_embedding, BaseLabel, SimpleLabel, MeanLabel
import torch
torch.cuda.set_device(device_num)

In [ ]:
#Mario vs Luigi Part 1
results_collection=[]

In [ ]:
def get_clip_logits_per_image(prompt, images):
    return [get_clip_logits(image, [prompt])[0] for image in images]

def random_colors(length=100):
    return [rp.random_rgb_float_color() for _ in range(length)]

def get_score(foreground, alpha, prompt:str, colors:list):
    
    alpha=alpha>.5
    rp.display_image(rp.blend_images(rp.random_rgb_float_color(),rp.cv_resize_image(foreground,rp.get_image_dimensions(alpha)),alpha))
    alpha=rp.cv_dilate(alpha,5,circular=True)
    
    assert rp.is_image(foreground) and rp.is_image(alpha)
    images = [rp.blend_images(foreground, color, alpha) for color in colors]
    scores = get_clip_logits_per_image(prompt, images)
    score = rp.mean(scores)
    return score

In [ ]:
def ranked_results(results):
    colors = random_colors()
    def score(result):
        output = get_score(foreground = result.image,
                         alpha = result.alphas[0],
                         prompt = result.p_name,
                         colors = colors,
                        )
        print(output)
        return output
    scores = list(map(score,results))
    scores, results = rp.sync_sort(scores, results)
    
    #First is best
    scores, results = scores[::-1], results[::-1]
    
    return scores, results

def display_ranked_results(scores, results):
    for score, result in zip(scores, results):
        rp.display_image(
            rp.labeled_image(
                rp.horizontally_concatenated_images(
                    rp.cv_resize_image(result.image, (256, 256)), result.alphas[0]
                ),
                "%s : %f" % (result.p_name, score),
            )
        )

In [ ]:
thumbnails=[]
def get_result_thumbnail(results):
    alpha, image = results.alphas[0], results.image
    height = width = 512 
    image = rp.cv_resize_image(image, (height, width))
    alpha = rp.cv_resize_image(alpha, (height, width),interp='nearest')
    method = " + ".join(
        [
            *(["CLIP"] if results.clip_coef else []),
            *(["StableDreamLoss"] if results.use_stable_dream_loss else []),
        ]
    )
    path = rp.get_relative_path(results.output_folder)
    name = results.p_name
    settings=[
        'representation',
        'LEARNING_RATE',
        'NUM_ITER',
        'GRAVITY',
        'clip_coef',
        'use_stable_dream_loss',
        'GUIDANCE_SCALE',
        'min_step',
        'max_step',
        'clip_coef',
    ]
    settings=[x+': '+str(results[x]) for x in settings]
    
    
    text=rp.line_join([name, ' ', method, ' ', path, '', 'Settings:',*settings])
    text=rp.wrap_string_to_width(text,60)
    text = rp.cv_text_to_image(text,monospace=False)
    text = rp.resize_image_to_fit(text, height, width)
    image=rp.horizontally_concatenated_images(image,alpha,text)
    
    out_dir='thumbnails_data6'
    rp.make_directory(out_dir)
    out_name='%s ____ %s ____ %i.png'%(method, name,len(rp.get_all_files(out_dir)))
    out_path=rp.path_join(out_dir,out_name)
    rp.save_image(image,out_path)
    
    thumbnails.append(image)
    rp.display_image(image)
    return image

In [ ]:
experiment_input_data = [
    [ [x.prompt], [x.path] , x] for x in data.samples
]

In [ ]:
experiment_setting_presets = {
    'clip_raster_bilateral':dict(
        representation="raster bilateral",
        LEARNING_RATE=1e-0,
        # NUM_ITER=500,
        GRAVITY=0.05,
        clip_coef=500,
        use_stable_dream_loss=False,
    ),
    'raster_bilateral':dict( #This one is also good!
        representation="raster bilateral",
        LEARNING_RATE=1e-0,
        GUIDANCE_SCALE=200,
        # NUM_ITER=500,
        GRAVITY=0.05,
        # min_step=10,
        # max_step=600,
    ),
    # dict(
    #     representation="raster bilateral",
    #     LEARNING_RATE=1e-0,
    #     GUIDANCE_SCALE=200,
    #     # NUM_ITER=500,
    #     GRAVITY=0.05,
    #     min_step=200,
    #     max_step=400,
    # ),
    # dict(
    #     representation="raster bilateral",
    #     LEARNING_RATE=1e-0,
    #     # NUM_ITER=500,
    #     GRAVITY=0.05,
    #     clip_coef=500,
    #     use_stable_dream_loss=True,
    #     GUIDANCE_SCALE=200,
    #     min_step=10,
    #     max_step=600,
    # ),
    # dict(NUM_ITER=500),
    'pure_fourier':dict(representation="fourier"),
    'default':dict(),
    'pure_raster':dict(representation="raster", LEARNING_RATE=1, GUIDANCE_SCALE=200,GRAVITY=.1),
}

In [ ]:
# experiment_setting_presets=[dict()]

things_to_try=[]

for prompts, urls, sample in rp.shuffled(experiment_input_data):
    for prompt in prompts:
        for url in urls:
            things_to_try.append([prompt,url,sample])

i=0
while True:
    # things_to_try=rp.shuffled(things_to_try)
    for prompt,url,sample in things_to_try:
        sample_name=sample.name
        for preset_name,preset in experiment_setting_presets.items():
            i+=1
            rp.fansi_print('EXPERIMENT NUMBER: %i'%i,'green','bold')
            rp.fansi_print(preset_name,'green','bold')
            rp.fansi_print(sample_name,'green','bold')

            rp.ic(prompt,url,preset)

            results = run_peekaboo(
                name=sample_name+'.'+prompt+'.'+preset_name,
                label=SimpleLabel(prompt),
                image=url,
                **preset,
            )

            get_result_thumbnail(results)

# OTHER STUFF

In [ ]:
results=run_peekaboo(
    # 'Green Luigi',
    # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",


    # 'orange basketball',
    # 'brown american football',
    # 'black and white soccer ball on the bottom right',
    # 'https://m.media-amazon.com/images/I/711pmt372VL.jpg',
    
    'Beverly Crusher',
    # 'https://www.google.com/url?sa=i&url=https%3A%2F%2Fmemory-alpha.fandom.com%2Fwiki%2FBeverly_Crusher&psig=AOvVaw2_2z0ZVduCSQH1zpoNfPlY&ust=1675160497935000&source=images&cd=vfe&ved=0CA8QjRxqFwoTCNDkx6qJ7_wCFQAAAAAdAAAAABAE',
    'https://imgix.bustle.com/uploads/image/2020/6/30/5a0a61e2-5518-4bb9-b8ea-66e3b3986ad7-picard-cruhser.jpg?w=349&h=262&fit=max&auto=format%2Ccompress',
    
    # 'Harry Potter boy - daniel radcliffe - man with glasses',
    # 'Hermoine Granger - Emma Watson - Girl with long hair',
    # 'Hermoine',
     # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
     # "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
    
    # 'Dory the fish',
    # 'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',

    # 'brass shiny saxophone on top',
     # "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",

    #Just CLIP
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.15, clip_coef=5000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.05, clip_coef=3000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=False, #BEST
    
    #Just DL
     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05, min_step=10, max_step=600,

    #Both
    representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=True, GUIDANCE_SCALE=200, min_step=10, max_step=600,
)

In [ ]:
results.output_folder

In [ ]:
#Mario vs Luigi Part 1
# peekaboo.s.min_step, peekaboo.s.max_step=200,600
results=run_peekaboo(
    # 'Green Luigi',
    # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",


    # 'orange basketball',
    # 'brown american football',
    'black and white soccer ball on the bottom right',
    'https://m.media-amazon.com/images/I/711pmt372VL.jpg',
    
    # 'Harry Potter boy - daniel radcliffe - man with glasses',
    # 'Hermoine Granger - Emma Watson - Girl with long hair',
    # 'Hermoine',
     # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
     # "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
    
    # 'Dory the fish',
    # 'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',

    # 'brass shiny saxophone on top',
     # "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",

    #Just CLIP
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.15, clip_coef=5000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-1, NUM_ITER=500,  GRAVITY=.05, clip_coef=3000, use_stable_dream_loss=False,
    # representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=False, #BEST
    
    #Just DL
     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05, min_step=10, max_step=600,

    #Both
    representation='raster bilateral', LEARNING_RATE=1e-0, NUM_ITER=500,  GRAVITY=.05, clip_coef=500, use_stable_dream_loss=True, GUIDANCE_SCALE=200, min_step=10, max_step=600,
)

In [ ]:
peekaboo.s.min_step=200
peekaboo.s.max_step=600

#Mario vs Luigi Part 1
results=run_peekaboo(
    # 'Luigi', #200/600
    # 'Princess Peach', #200/600H, 
    # 'Mario',
    # 'Bowser from super smash bros; yellow shell; seen from the front', #NO; stable diffusion is too weak to draw this - or I have bad prompts
    
    
                     # "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     # "http://images6.fanpop.com/image/photos/43400000/Mario-Peach-super-mario-bros-43431011-644-704.png",
                                         
                     # bilateral_kwargs=dict(kernel_size = 3,
                     #                  tolerance = .08,
                     #                  sigma = 5,
                     #                  iterations=2,
                     #                 ),
                     # representation='raster',
    
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
for _ in range(10):
    results=run_peekaboo('Luigi',
                         "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                         representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, GRAVITY=.05,
                         min_step=200,
                         max_step=400,
                         NUM_ITER=100,
                         )
    results_collection.append(results)



In [ ]:
#Mario vs Luigi Part 2
results=run_peekaboo('Luigi',
                     "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     )

In [ ]:
#Mario vs Luigi Part 2
label = peekaboo.SimpleLabel('Luigi')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Mario').embedding*blend_factor
results=run_peekaboo('Luigi',
                     "https://i1.sndcdn.com/artworks-000160550668-iwxjgo-t500x500.jpg",
                     label=label
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 1
results=run_peekaboo('Wine Glass half-full',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 1
results=run_peekaboo('Wine Glass half-full',
                     'Swiss Cheese',
                     'Grapes',
                     'Green Grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 2
results=run_peekaboo('Swiss Cheese',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 2
results=run_peekaboo('Swiss Cheese',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
# https://americasrestaurant.com/wp-content/uploads/2022/04/what-do-bread-and-butter-pickles-taste-like.jpg
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 3
results=run_peekaboo(
    # 'piano keyboard',
    # 'violin',
    'brass shiny saxophone on top',
                     "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",
                     # representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
for _ in range(10):
    results=run_peekaboo(
        'Nemo Clownfish',
        'https://akns-images.eonline.com/eol_images/Entire_Site/2016917/rs_600x600-161017135949-600.finding-dory-2.101716.jpg',
        representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200,
        min_step=200, max_step=600,
        GRAVITY=.05,
        NUM_ITER=200,
    )
    results_collection.append(results)

In [ ]:
for _ in range(10):
    results=run_peekaboo(
        'brass shiny saxophone on top',
        "https://c8.alamy.com/comp/2GFDH27/violin-with-saxophone-and-piano-closeup-2GFDH27.jpg",
        representation='raster bilateral',
        LEARNING_RATE=1e-0,
        GUIDANCE_SCALE=200,
        min_step=200,
        max_step=900,
        GRAVITY=.05,
        NUM_ITER=200,
    )
    results_collection.append(results)

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,400
#Wine and Cheese Part 3
results=run_ipeekaboo('grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
peekaboo.s.min_step,peekaboo.s.max_step=200,600
#Wine and Cheese Part 3
results=run_peekaboo('Green grapes',
                     "https://emeraldvalleyartisans.com/wp-content/uploads/2015/10/winecheese1.jpg",
                     representation='raster bilateral', LEARNING_RATE=1e-0, GUIDANCE_SCALE=200, NUM_ITER=500,  GRAVITY=.05,
                     )

In [ ]:
#Harry Potter Part 1
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     NUM_ITER=300
                    )

In [ ]:
#Harry Potter Part 2
label = peekaboo.SimpleLabel('Hermoine Granger')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Harry Potter').embedding*blend_factor
results=run_peekaboo('Hermoine',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     label=label
                     )


In [ ]:
#Harry Potter Part 2
label = peekaboo.SimpleLabel('Hermoine Granger - Emma Watson - Girl with long hair')
blend_factor=.5
label.embedding = label.embedding*(1+blend_factor) - peekaboo.SimpleLabel('Harry Potter boy - daniel radcliffe - man with glasses').embedding*blend_factor

results=run_peekaboo('Hermoine',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     label=label
                     )

In [ ]:
#Emma Watson
#Good in a row: |||
#Bad prompts: "Hermoine Granger aka Emma Watson"
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://cometothedarkside97.files.wordpress.com/2021/05/images-2020-01-25t073939.551.jpeg?w=468",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Emma Watson
#Good in a row: 
#Bad prompts: 
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://prisiliaandayani.files.wordpress.com/2014/03/harry-ron-hermione-harry-potter-2422078-361-500.jpg",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Good in a row with fixed timestep 500: H,H,
#Bad prompts: 

# peekaboo.s.min_step=100
# peekaboo.s.max_step=500
for _ in range(10):
    results=run_peekaboo(
        'Harry Potter boy - daniel radcliffe - man with glasses',
        # 'Hermoine Granger - Emma Watson - Girl with long hair',

                         "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
                            # "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                         # "https://prisiliaandayani.files.wordpress.com/2014/03/harry-ron-hermione-harry-potter-2422078-361-500.jpg",

                         representation='raster bilateral',
                         # representation='raster',
                         # bilateral_kwargs=dict(kernel_size = 3,
                         #                  tolerance = .08,
                         #                  sigma = 5,
                         #                  iterations=2,
                         #                 ),
                         # representation='raster',
                         LEARNING_RATE=1e-0,
                         GUIDANCE_SCALE=200,
                         min_step=200,
                         max_step=400,
                         GRAVITY=.05,
                         NUM_ITER=200,
                         )
    results_collection.append(results)

In [ ]:
#Emma Watson
#Good in a row: ||X2X
#Good in a row with fixed timestep: |||||
#Bad prompts: 
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://i.guim.co.uk/img/media/380500f7fd9321a6cfd1bd1a3b3f104ad9797bb7/0_0_2000_1199/master/2000.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=f0d0ecb5a0c33aa91be270b08619e8d7",
                     representation='raster bilateral',
                     # bilateral_kwargs=dict(kernel_size = 3,
                     #                  tolerance = .08,
                     #                  sigma = 5,
                     #                  iterations=2,
                     #                 ),
                     # representation='raster',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Emma Watson
#Good in a row: |||
#Bad prompts: "Emma Watson"
results=run_peekaboo('Hermoine Granger aka Emma Watson',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter 
#Good in a row: ||
results=run_peekaboo('Daniel radcliffe',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter 
#Good in a row: |||||
results=run_peekaboo('Harry Potter boy - daniel radcliffe - man with glasses',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Mario vs Luigi Part 1
#Good in a row: |||||
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     representation='raster bilateral',
                     LEARNING_RATE=1e-0,
                     GRAVITY=.05,
                     )

In [ ]:
#Harry Potter Part 2
results=run_peekaboo('Hermoine Granger - Emma Watson - Girl with long hair - left',
                     "https://assets.teenvogue.com/photos/569e7d2a74da98670ff0ce1c/1:1/w_2159,h_2159,c_limit/MCDHAPO_EC797_H.JPG",
                     NUM_ITER=300
                    )

In [ ]:
#Stable Diffusion Basic Demo
#This is a demo cell showing how to simply use Stable Diffusion to generate an image. This is usefull for prompt engineering.
rp.display_image(peekaboo.s.prompt_to_img('Cake'))

In [ ]:
#Stable Diffusion Basic Demo
#This is a demo cell showing how to simply use Stable Diffusion to generate an image. This is usefull for prompt engineering.
for _ in range(30):
    rp.display_image(peekaboo.s.prompt_to_img('Bowser from super smash bros; yellow shell; seen from the front'))

In [ ]:
rp.local_copy(results)

In [ ]:
results